# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine, text
from nltk.tokenize import word_tokenize
import nltk
nltk.download(['punkt', 'wordnet','punkt_tab' ])

from nltk.stem import WordNetLemmatizer


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\johannes.misensky\AppData\Roaming\nltk_data..
[nltk_data]     .
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\johannes.misensky\AppData\Roaming\nltk_data..
[nltk_data]     .
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\johannes.misensky\AppData\Roaming\nltk_data..
[nltk_data]     .
[nltk_data]   Package punkt_tab is already up-to-date!


In [2]:
from sklearn.pipeline import Pipeline
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [14]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
with engine.connect() as conn:
    # Use the connection to execute the query and read into a DataFrame
    df = pd.read_sql(text("SELECT * FROM Message"), conn)

X = df['message'].values
Y = df.iloc[:, 6:].values



### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([('cV', CountVectorizer(tokenizer=tokenize)),
                      ('tfidf', TfidfTransformer()),
                      ('multClas', MultiOutputClassifier(RandomForestClassifier()))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
pipeline.fit(X_train, y_train)

Pipeline(steps=[('cV',
                 CountVectorizer(tokenizer=<function tokenize at 0x0000024DC4EFEC10>)),
                ('tfidf', TfidfTransformer()),
                ('multClas',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

In [7]:
y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
len(df.columns[6:])

35

In [13]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred, target_names= df.columns[6:]))

                        precision    recall  f1-score   support

               request       0.89      0.41      0.56      1149
                 offer       0.00      0.00      0.00        32
           aid_related       0.81      0.55      0.65      2703
          medical_help       0.95      0.04      0.07       505
      medical_products       0.76      0.07      0.13       315
     search_and_rescue       0.78      0.03      0.07       203
              security       1.00      0.01      0.02       127
              military       0.67      0.03      0.05       211
           child_alone       0.00      0.00      0.00         0
                 water       0.91      0.29      0.44       395
                  food       0.85      0.52      0.64       724
               shelter       0.83      0.35      0.49       552
              clothing       0.64      0.07      0.12       103
                 money       0.57      0.03      0.05       145
        missing_people       1.00      

C:\Users\johannes.misensky\Anaconda3\envs\test\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\johannes.misensky\Anaconda3\envs\test\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\johannes.misensky\Anaconda3\envs\test\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result

### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
print(pipeline.get_params())

{'memory': None, 'steps': [('cV', CountVectorizer(tokenizer=<function tokenize at 0x0000024DC4EFEC10>)), ('tfidf', TfidfTransformer()), ('multClas', MultiOutputClassifier(estimator=RandomForestClassifier()))], 'verbose': False, 'cV': CountVectorizer(tokenizer=<function tokenize at 0x0000024DC4EFEC10>), 'tfidf': TfidfTransformer(), 'multClas': MultiOutputClassifier(estimator=RandomForestClassifier()), 'cV__analyzer': 'word', 'cV__binary': False, 'cV__decode_error': 'strict', 'cV__dtype': <class 'numpy.int64'>, 'cV__encoding': 'utf-8', 'cV__input': 'content', 'cV__lowercase': True, 'cV__max_df': 1.0, 'cV__max_features': None, 'cV__min_df': 1, 'cV__ngram_range': (1, 1), 'cV__preprocessor': None, 'cV__stop_words': None, 'cV__strip_accents': None, 'cV__token_pattern': '(?u)\\b\\w\\w+\\b', 'cV__tokenizer': <function tokenize at 0x0000024DC4EFEC10>, 'cV__vocabulary': None, 'tfidf__norm': 'l2', 'tfidf__smooth_idf': True, 'tfidf__sublinear_tf': False, 'tfidf__use_idf': True, 'multClas__estimato

In [11]:
from sklearn.model_selection import GridSearchCV
parameters = {
    'cV__max_df': [0.75, 1.0],  # max_df: Ignore terms that appear in more than x% of the documents
    'cV__ngram_range': [(1, 1), (1, 2)],  # n-grams: Unigrams or bigrams
    'tfidf__use_idf': [True, False],  # Use or not use IDF
    'multClas__estimator__n_estimators': [10, 50, 100],  # Number of trees in the forest (RandomForest)
    'multClas__estimator__min_samples_split': [2, 4, 6],  # Minimum number of samples required to split a node
}


cv = GridSearchCV(pipeline, parameters, cv=3, verbose=3, n_jobs=-1)

cv.fit(X_train, y_train)  # Assuming you have already split your data into X_train and Y_train

# Print the best parameters
print("Best Parameters: ", cv.best_params_)

# Evaluate the best estimator on test data
best_model = cv.best_estimator_
y_pred = best_model.predict(X_test)

Fitting 3 folds for each of 72 candidates, totalling 216 fits
Best Parameters:  {'cV__max_df': 1.0, 'cV__ngram_range': (1, 2), 'multClas__estimator__min_samples_split': 2, 'multClas__estimator__n_estimators': 100, 'tfidf__use_idf': False}


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
import pickle

with open("best_model.pkl", "wb") as f:
    pickle.dump(best_model, f)

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.